웹 크롤링
=========

*how to?*
---------

> ### **1.필요한 모듈 불러오기**
>>     from bs4 import BeautifulSoup , import requests 
> ### **2. html 구조 이해하기**
>> * mac-> 개발자 도구 활성화 - 페이지 소스 보기/ 윈도우 - F12
>> * <여는태그 속성명="속성값">내용</닫는태그>
>> * <태그 class = "클래스명" > 
>>> ex) div : 구역 표시하는 태그, p : 문단 표시하는 태그, body : 실제적으로 보여지는 부분 나타내는 태그...

> ### **3.텍스트 잘 뽑아오기**
>>    위치 정확히 지정, 클래스나 태그 명 잘 보기.
>>>     find(id='id명')
    findAll('tag') or find_all('tag)
    findAll('tag', {class:'class'})
    find('class')
    select('위계구조')
    select_one('원하는 것')
    
>>>>**Select 와 find의 차이점**
>>>>>select finds multiple instances and returns a list, find finds the first, so they don't do the same thing. select_one would be the equivalent to find.
    
>>>> 
    get_text()
    text()
    next_sibling : 텍스트 노드인 경우



### 1. IDMB 영화 리뷰 크롤링

In [1]:
# 1. 웹크롤링에 필요한 2가지 모듈 호출
from urllib.request import urlopen # 특정 웹서버에 접근
from bs4 import BeautifulSoup # 웹페이지 내용구조 해석
import requests #파이썬에서 http 요청 보내는 라이브러리 
from urllib import parse
from bs4 import BeautifulSoup #html 구조에서 간딴히 정보 빼내기 위해 사용하는 라이브러리

#url 지정
url= 'https://www.imdb.com/title/tt4154756/reviews?ref_=tt_ql_3'
url  = url.format(1)

#webpage에 urlopen으로 열어줘서 할당
res = requests.get(url)


#source 파일에....
source = BeautifulSoup(res.text, 'html.parser')

In [2]:
# #자바html 형식으로 되어있음.
# source

In [3]:
review_list = source.find_all('div', {'class': 'imdb-user-review'})
review_p = source.p.string
review_list2 = source.find_all(['a','span'])
review_only =  source.find_all('div',{'class': "text show-more__control"})

In [26]:
review_list[0]

<div class="lister-item mode-detail imdb-user-review collapsable" data-review-id="rw5504583" data-vote-url="/title/tt4154756/review/rw5504583/vote/interesting">
<div class="review-container">
<div class="lister-item-content">
<div class="ipl-ratings-bar">
<span class="rating-other-user-rating">
<svg class="ipl-icon ipl-star-icon " fill="#000000" height="24" viewbox="0 0 24 24" width="24" xmlns="http://www.w3.org/2000/svg">
<path d="M0 0h24v24H0z" fill="none"></path>
<path d="M12 17.27L18.18 21l-1.64-7.03L22 9.24l-7.19-.61L12 2 9.19 8.63 2 9.24l5.46 4.73L5.82 21z"></path>
<path d="M0 0h24v24H0z" fill="none"></path>
</svg>
<span>9</span><span class="point-scale">/10</span>
</span>
</div>
<a class="title" href="/review/rw5504583/"> Great
</a> <div class="display-name-date">
<span class="display-name-link"><a href="/user/ur87850731/">MR_Heraclius</a></span><span class="review-date">23 February 2020</span>
</div>
<div class="content">
<div class="text show-more__control">This movie portrays

In [4]:
#화면 속 글자만 가져오고 싶다!!
# for review in review_list:
#     print(review.get_text().strip())

In [5]:
# lis = source.select('div.lister-list > div.imdb-user-review')

# for i in range(len(lis)):
#     score = lis[i].find('span').text.strip()[0]
#     text = lis[i].find('div',{'class':'text show-more__control'}).text.strip()
#     print('score :' + score)
#     print('    ')
#     print('comment: ' + text)
#     print('    ')

## *코로나 현황 크롤링 실습* 
간단 ver.

In [6]:
# 오늘의 코로나 현황

webpage = urlopen('http://ncov.mohw.go.kr/') #웹페이지 url 끌어오기
source_corona = BeautifulSoup(webpage, 'html.parser',from_encoding='utf-8') #html.parser 이용하여 html 구조 가져오기

In [7]:
# source_corona

In [8]:
#웹페이지 개발자 도구 (윈도우는 f12, 맥은 사파리 설정)
#원하는 부분의 <A class =  B> 
#변수 = 위의 source.findAll( A, {'class':'B'})

div = source_corona.findAll('span',{'class': 'data'}) 

In [9]:
#html의 구조 get
div

[<span class="data">833</span>, <span class="data">37</span>]

In [10]:
#여기서 필요한 텍스트만 뽑아볼 것임
for a in div:
    print(a.get_text().strip())

833
37


In [11]:
#리스트화 시켜서 프린트 할 때 편하게...
list = []
for a in div:
    list.append(a.get_text())

In [12]:
list

['833', '37']

In [13]:
print('>> #오늘의 #국내 #코로나19 #현황 \n')
print('국내 유입:' ,list[0], '해외유입:' , list[1])

>> #오늘의 #국내 #코로나19 #현황 

국내 유입: 833 해외유입: 37


*여러 페이지에 있는 리뷰 크롤링하고, 파일로 저장하기*
========

> ### 1. 제목, 평점, 리뷰 들어있는 csv 만들어보기 
>>#### 네이버 평점, 리뷰 칸 속 평점.리뷰 들어가보면 여러개의 리뷰들이 나옴.
>### 2. html의 구조를 살펴보면
>>#### table.list_netizen > tbody > tr > td.title 속에 내용 다 있음.
>>>#### 제목 => a.movie.color_b
>>>#### 평점 => div.list_netizen_score > em
>>>#### 리뷰 => br 하위의 텍스트를 반환해야하는 것. => br.next_sibling 사용.

In [14]:
import requests #파이썬에서 http 요청 보내는 라이브러리 
from urllib import parse
from bs4 import BeautifulSoup #html 구조에서 간딴히 정보 빼내기 위해 사용하는 라이브러리
url = 'https://movie.naver.com/movie/point/af/list.nhn?&page=1'
res = requests.get(url)

soup = BeautifulSoup(res.text,'lxml')
tds = soup.select('table.list_netizen > tbody > tr > td.title')
print(len(tds))


10


In [15]:
# tds

In [16]:
comment_list=[]
for td in tds: 
    movie_title = td.find('a').text.strip()
    link = td.select_one('a.movie.color_b').get('href')
    link = parse.urljoin('https://movie.naver.com/movie/point/af/list.nhn?&page=2', link)
    score = td.select_one('div.list_netizen_score > em ').text.strip()
    comment = td.select_one('br').next_sibling.strip()
    comment_list.append((movie_title,score, comment))
    
    print(movie_title, score, comment, sep=" / ")

마라가 큰 결정을 해야 해 / 10 / 오늘 볼거지롱 딱봐도 잼날거 같지 않음?
미스터 빈의 홀리데이 / 10 / 시대를 타지 않는 명화~
조작된 도시 / 10 / 왜 이걸 극장에서 못봤지.
나이브스 아웃 / 10 / 최고로 재미있던 추리 영화
걸 / 10 / 부국제에서 보고 ㅠㅜ 감동받았는데 드디어개봉하는구나,
걸 / 10 / 정말 감동적인 영화!!
달마야, 서울 가자 / 10 / 
국제수사 / 1 / 평점은 역시 다음이daum 정확하다 네이버는 존나 후하네
걸 / 10 / 
이웃사촌 / 1 / 진짜 삼각하게 재미없네


In [17]:
import pandas as pd

df = pd.DataFrame(comment_list, columns=['제목','평점','리뷰'])
df.to_csv('naver_comment.csv', encoding= 'utf-8', index=False)

### 만약 두번째 페이지도 이어서 크롤링하고 싶다면? For 문을 사용하자!

https://movie.naver.com/movie/point/af/list.nhn?&page=1


https://movie.naver.com/movie/point/af/list.nhn?&page=2


이런식으로 페이지 뒷부분에 숫자의 변화가 생김.

In [18]:
base_url = 'https://movie.naver.com/movie/point/af/list.nhn?&page={}'
comment_list=[]
import time
import random

for page in range (10):
    url = base_url.format(page)
    res = requests.get(url)
    
    soup = BeautifulSoup(res.text,'lxml')
    tds = soup.select('table.list_netizen > tbody > tr > td.title')
    
    for td in tds: 
        movie_title = td.find('a').text.strip()
        link = td.select_one('a.movie.color_b').get('href')
        link = parse.urljoin('https://movie.naver.com/movie/point/af/list.nhn?&page=2', link)
        score = td.select_one('div.list_netizen_score > em ').text.strip()
        comment = td.select_one('br').next_sibling.strip()
        comment_list.append((movie_title,score, comment))

    interval = round(random.uniform(0.2,1),2)
    time.sleep(interval)

print('종료')

종료


In [19]:
df = pd.DataFrame(comment_list, columns=['제목','평점','리뷰'])
df.to_csv('naver_comment.csv', encoding= 'utf-8', index=False)

In [20]:
df

,제목,평점,리뷰
0,마라가 큰 결정을 해야 해,10,오늘 볼거지롱 딱봐도 잼날거 같지 않음?
1,미스터 빈의 홀리데이,10,시대를 타지 않는 명화~
2,조작된 도시,10,왜 이걸 극장에서 못봤지.
3,나이브스 아웃,10,최고로 재미있던 추리 영화
4,걸,10,"부국제에서 보고 ㅠㅜ 감동받았는데 드디어개봉하는구나,"
...,...,...,...
95,클로젯,7,햐. 감동을 원하냐? 공포를 원하냐? 뭐냐
96,"마 레이니, 그녀가 블루스",7,절반 이상이 지하 연습실에서 밴드의 대화지만 음악은 훌륭하고 채드윅의 분노가 잘 녹아있는
97,악마는 사라지지 않는다,10,인간의 욕망과 자의적인 종교적 행태가 낳은 인간의 파멸을 아주 섬세하게 다룬 드라마.
98,원더 우먼 1984,1,심형래 영화인줄 유치해서 버티기 힘들었다
